# Introduction
This notebook contains code for reading weather forecast predictions made using the GNN model of [Forecasting Global Weather with Graph Neural Networks](https://arxiv.org/abs/2202.07575), Keisler 2022.

The predictions were made for the year 2020, which the model was not trained or validated on, using ERA5 initializations.

### Imports

In [1]:
import zarr
import dask.array
import xarray as xr
import pandas as pd
import numpy as np

### Lazy-load the `zarr` array into a `dask` array.

In [2]:
gs_path = 'gs://rk-public-auto/wb2/keisler22_2020_predict'
ztmp = zarr.open(gs_path, mode='r')
data = dask.array.from_zarr(ztmp['data'])

### Wrap with an `xarray.DataArray` containing all dimensions and coordinates.

In [3]:
# define the forecast initialization datetimes.
init = pd.date_range(start='2020-01-01 00:00:00', end='2020-12-31 12:00:00', freq='12H')

# define the forecast hour coordinates.
# fhour=0 corresponds to the ERA5 initialization.
# there are 40 6-hour steps after the initialization.
fhour = np.arange(0, 40+1)*6

# define the lat/lon coordinates.
lat = np.arange(+90, -90-1, -1.0)
lon = np.arange(0, 360, +1.0)
lon[lon>180] -= 360

# define the bands, i.e. channels.
band = [
    'z500', 'z700','z850',
    't500', 't700', 't850',
    'q500', 'q700', 'q850',
    'u500', 'u700', 'u850',
    'v500', 'v700', 'v850',
]

# check that the shape of the data array matches the shape of these coordinate arrays.
assert data.shape == (len(init), len(fhour), len(lat), len(lon), len(band))

# create the xr.DataArray.
da = xr.DataArray(
    data,
    name='data',
    dims=('init','fhour','lat','lon','band'),
    coords={
        'init': init,
        'fhour': fhour,
        'lat': lat,
        'lon': lon,
        'band': band,
    },
)

In [4]:
da

<xarray.DataArray 'data' (init: 732, fhour: 41, lat: 181, lon: 360, band: 15)>
dask.array<from-zarr, shape=(732, 41, 181, 360, 15), dtype=float32, chunksize=(1, 1, 181, 360, 15), chunktype=numpy.ndarray>
Coordinates:
  * init     (init) datetime64[ns] 2020-01-01 ... 2020-12-31T12:00:00
  * fhour    (fhour) int64 0 6 12 18 24 30 36 42 ... 204 210 216 222 228 234 240
  * lat      (lat) float64 90.0 89.0 88.0 87.0 86.0 ... -87.0 -88.0 -89.0 -90.0
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... -5.0 -4.0 -3.0 -2.0 -1.0
  * band     (band) <U4 'z500' 'z700' 'z850' 't500' ... 'v500' 'v700' 'v850'